In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import gmaps
import requests
import json
from config import gkey

In [ ]:
file = "DataForTable2.1.csv"
happiness_years = pd.read_csv(file)
happiness_years.head()

In [ ]:
happiness_years.describe()

In [ ]:
happiness_years.loc[happiness_years["Generosity"] == 0]

In [ ]:
# keep needed series
happiness_2021 = happiness_years.loc[happiness_years['year']==happiness_years['year'].max(),
                                   ["Country name","Life Ladder","Log GDP per capita","Social support",
                                   "Healthy life expectancy at birth","Freedom to make life choices"]]
happiness_2021["Country name"].nunique()

In [ ]:
# HOW TO HANDLE NaN VALUES - remove null values for any of the 4 colums we're looking at

#set NaN values to 0 - none of the colums have a legitimate value of 0
#happiness_years = happiness_years.fillna(0)
#happiness_years.head()
happiness_2021 = happiness_2021.dropna()
happiness_2021.head()

In [ ]:
happiness_2021 = happiness_2021.rename(columns = {'Log GDP per capita': 'GDP per capita',
                                              'Life Ladder':'Happiness score',
                                             'Healthy life expectancy at birth': 'healthy life expectancy',
                                                 'Country name':'Country'})
happiness_2021.head()

In [ ]:
#corr table for reference
corr = happiness_2021.corr()
corr = corr.style.background_gradient(cmap='PRGn')
corr

In [ ]:
#adding latitude and longitude for each country
gkey = gkey
lat = []
lng = []
for country in happiness_2021["Country"]:
    target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={country}&components=country:{country}&key={gkey}"
    # components=country:{country} makes sure the country of Georgia is mapped correctly!
    response = requests.get(target_url).json()
    lat.append(response['results'][0]['geometry']['location']['lat'])
    lng.append(response['results'][0]['geometry']['location']['lng'])
happiness_2021['Latitude'] = lat
happiness_2021["Longitude"] = lng

In [ ]:
#export to csv
happiness_2021.to_csv('cleaned_happiness_2021.csv', index = False)

In [ ]:
# Start of Amanda's code

In [ ]:
# re-importing cleaned data so as to not have to rerun above code
happiness_2021 = pd.read_csv('cleaned_happiness_2021.csv')
happiness_2021.describe()

### Social Support
Per the World Happiness Report: 
>Social support (or having someone to count on in times of trouble) is the national average of the binary responses (either 0 or 1) to the GWP question “If you were in trouble, do you have relatives or friends you can count on to help you whenever you need them, or not?”


### Null Hypothesis Test
Null hypothesis - the means for happiness of the top, middle, and bottom third of countries based on social support are the same

In [ ]:
# sort dataset and see how many values need to be in each group
h0_test = happiness_2021[["Country","Happiness score","Social support"]]
h0_test = h0_test.sort_values("Social support")
size = h0_test.count() # 110 countries total
# size/3 = 36.666667 - make 2 groups at 37 and one at 36
h0_test.head()

In [ ]:
bottom_third = h0_test.iloc[0:37,:]
bottom_third

In [ ]:
middle_third = h0_test.iloc[37:74,:]
middle_third

In [ ]:
top_third = h0_test.iloc[74:110,:]
top_third

In [ ]:
# concatenate the thirds back together just to triple check there aren't duplicate countries
check_df = pd.concat([top_third,middle_third,bottom_third])
check_df.value_counts()

In [ ]:
# get only the happiness score for each group and double check the count is correct
top_happiness = top_third["Happiness score"]
middle_happiness = middle_third["Happiness score"]
bottom_happiness = bottom_third["Happiness score"]
total = top_happiness.count() + middle_happiness.count() + bottom_happiness.count()
print(total)

In [ ]:
# run ANOVA test!
st.f_oneway(top_happiness,middle_happiness,bottom_happiness)

In [ ]:
bottom_third['Social support'].std()

In [ ]:
middle_third['Social support'].std()

In [ ]:
top_third['Social support'].std()

In [ ]:
# Run Kruskal test as standard deviations might be different
st.kruskal(top_happiness,middle_happiness,bottom_happiness)

### Result
The pvalue is either 3.0050980152485063e-22 or 8.261872947301994e-16 (depending on test), both of which are quite smaller than 0.5! This rejects the null hypothesis that says the means of all three groups are the same, indicating there is a statistically significant correlation between happiness and social support.

### Other analysis

In [ ]:
x_values = happiness_2021['Social support']
y_values = happiness_2021['Happiness score']

In [ ]:
quartiles = x_values.quantile([0.25,0.5,0.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq
lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Q1: {lowerq}\nQ3: {upperq}\nIQR: {iqr}\nLower Bound: {lower_bound}\nUpper Bound: {upper_bound}")

In [ ]:
outliers = happiness_2021.loc[x_values<lower_bound,:]
outliers
#outliers.sort_values('Social support')

In [ ]:
plt.boxplot(x_values, showmeans=True)
plt.title("Social Support Values")
for country in outliers['Country'].values:
    y = outliers.loc[outliers['Country']==country,'Social support']
    plt.annotate(country,(1.05,y),fontsize=10,color="red")
plt.show()

In [ ]:
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

In [ ]:
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(0.7,2.4),fontsize=15,color="red")
plt.xlabel("Social Support")
plt.ylabel("Happiness Score")
plt.title("Social Support\nvs\nTotal Happiness Score")
plt.show()

In [ ]:
pr = round(st.pearsonr(x_values,y_values)[0],2)
print(f'The correlation between social support and happiness is {pr}, suggesting a strong link between the two factors.')

In [ ]:
bottom_8 = happiness_2021[["Country","Happiness score","Social support"]].sort_values('Social support').head(8)
bottom_8.sort_values(['Happiness score','Social support'])

In [ ]:
x_values = bottom_8['Social support']
y_values = bottom_8['Happiness score']
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(0.51,2.4),fontsize=15,color="red")
plt.xlabel("Social Support")
plt.ylabel("Happiness Score")
plt.title("Social Support\nvs\nTotal Happiness Score")
plt.show()
pr = round(st.pearsonr(x_values,y_values)[0],2)
print(f'The correlation between social support and happiness for bottom 8 countries is {pr}')

In [ ]:
top_ten = happiness_2021.sort_values(['Social support','Happiness score'], ascending = False).head(10)

In [ ]:
gmaps.configure(api_key = gkey)
fig = gmaps.figure()

social_sorted = happiness_2021.sort_values('Social support')
top_locations = social_sorted.iloc[74:110,[6,7]]
top_social = social_sorted.iloc[74:110,3]
middle_locations = social_sorted.iloc[37:74,[6,7]]
middle_social = social_sorted.iloc[37:74,3]
bottom_locations = social_sorted.iloc[0:37,[6,7]]
bottom_social = social_sorted.iloc[0:37,3]

all_locations = happiness_2021[["Latitude","Longitude"]]
all_social = happiness_2021['Social support']
                                        
fig = gmaps.figure()

symbols_top = gmaps.symbol_layer(top_locations, fill_color='#028833', stroke_color='#028833')
fig.add_layer(symbols_top)

symbols_middle = gmaps.symbol_layer(middle_locations, fill_color='blue', stroke_color='blue')
fig.add_layer(symbols_middle)

symbols_bottom = gmaps.symbol_layer(bottom_locations, fill_color='#E65300', stroke_color='#E65300')
fig.add_layer(symbols_bottom)

fig


In [ ]:
# End of Amanda's code

In [ ]:
# Start John's code

In [ ]:
# End John's code

In [ ]:
# Start Nathan's code

Before beginning any analysis, it's important to note that with GDP data, we dropped countries in the original data file that have "na" values in GDP per capita, all of these countries have "na" because the GDP per capita may be in some way hard to determine, therefore what remains should be truthful values that can all be used in analysis. For this reason, to look for outliers would be redundant, because it has effectively been done for us.

In [ ]:
x_axis = happiness_2021["GDP per capita"]
y_axis = happiness_2021["Happiness score"]


(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_axis, y_axis)
regress_values = x_axis * slope + intercept
line_eq = f"y = {round(slope, 2)}x + {round(intercept, 2)}"
rsquared = f"r squared = {round(rvalue * rvalue, 2)}"

plt.plot(x_axis, regress_values, color='red')
plt.annotate(line_eq, (7,7.1), fontsize=15, color="red")
plt.annotate(rsquared, (7,6.6), fontsize=15, color="red")

plt.title("GDP Per Capita vs Happiness")
plt.ylabel("Happiness")
plt.xlabel("GDP Per Capita")

plt.scatter(x_axis, y_axis)
plt.show()

This is a simple scatterplot of the data, along with a linear regression, showing a general trend where higher GDP Per Capita leads to a higher happiness score.

In [ ]:
happiness_2021["GDP per capita"].hist(bins=25)

plt.xlabel("GDP Per Capita")
plt.ylabel("Counts")
plt.title("Distribution of GDP Per Capita data")

plt.show()

This plot uses bins of the GDP data to visualize where the GDP is clustered. We can use this to approximate higher, lower, and middle thirds of the data.

In [ ]:
# Extract individual groups
group0 = happiness_2021[happiness_2021["GDP per capita"] < 8.75]["Happiness score"]
group1 = happiness_2021[happiness_2021["GDP per capita"].between(8.75, 10.25, inclusive='both')]["Happiness score"]
group2 = happiness_2021[happiness_2021["GDP per capita"] > 10.25]["Happiness score"]

# Perform the ANOVA
st.f_oneway(group0, group1, group2)

With a P value much lower than .05, we can reject the null hypothesis that GDP per capita has no effect on happiness score.

In [ ]:
# End Nathan's code

In [ ]:
# Start Joey's code

In [ ]:
# End Joey's code